In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
import re
import string


dir_path = '../1. Original Data'
save_path = '../2. Prepared Data'
train_pattern = '/TRAIN_BD_'
test_pattern = '/TEST_BD_'
file_name = ['CONSUMO.csv','DEM_TIME.csv','/20180924_user_train.csv']

times = np.unique(pd.read_csv(save_path+file_name[2]).TIME)
times = list(times)

['abc', 'america-rn', 'atletico-mg', 'atletico-pr', 'bahia', 'botafogo', 'bragantino', 'chapecoense', 'corinthians', 'cruzeiro', 'figueirense', 'flamengo', 'fluminense', 'goias', 'gremio', 'gremio-prudente', 'icasa', 'internacional', 'luverdense', 'nautico', 'palmeiras', 'paysandu', 'ponte-preta', 'portuguesa', 'santos', 'sao-paulo', 'sport', 'vasco', 'vitoria']


##  <u>Bases de Consumo</u>:

In [2]:
df_con_train = pd.read_csv(dir_path+train_pattern+file_name[0],low_memory=True)#,nrows=210000)
df_con_test = pd.read_csv(dir_path+test_pattern+file_name[0],low_memory=True)#,nrows=90000)
print(df_con_train.head())
print(df_con_train.head())
print(df_con_train.info())
print(df_con_test.info())

       KEY   tempo                                                url
0  1666965  1532.0  http://globoesporte.globo.com/mg/futebol/times...
1  2428750   965.0  https://globoesporte.globo.com/futebol/times/b...
2    23359   464.0  http://globoesporte.globo.com/pb/futebol/campe...
3  2451676  3286.0  http://globoesporte.globo.com/sp/santos-e-regi...
4   707389  1195.0  http://globoesporte.globo.com/futebol/times/pa...
       KEY   tempo                                                url
0  1666965  1532.0  http://globoesporte.globo.com/mg/futebol/times...
1  2428750   965.0  https://globoesporte.globo.com/futebol/times/b...
2    23359   464.0  http://globoesporte.globo.com/pb/futebol/campe...
3  2451676  3286.0  http://globoesporte.globo.com/sp/santos-e-regi...
4   707389  1195.0  http://globoesporte.globo.com/futebol/times/pa...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14174707 entries, 0 to 14174706
Data columns (total 3 columns):
KEY      int64
tempo    float64
url      objec

In [7]:
#FUNCAO SIMPLES DE EXTRACAO DE TIME DO URL
def url_proc_dashed(url):
    for time in times:
        if 'times/'+time+'/' in str(url): 
            return time
        
df_con_train_proc_simple = df_con_train.copy()
df_con_train_proc_simple.url = df_con_train.url.apply(url_proc_dashed)
df_con_train_proc_simple.dropna(inplace=True)
print(df_con_train_proc_simple.head(5))

df_con_test_proc_simple = df_con_test.copy()
df_con_test_proc_simple.url = df_con_test.url.apply(url_proc_dashed)
df_con_test_proc_simple.dropna(inplace=True)
print(df_con_test_proc_simple.head(5))

       KEY   tempo        url
1  2428750   965.0   botafogo
3  2451676  3286.0     santos
4   707389  1195.0  palmeiras
7  1317005   468.0   cruzeiro
8   926362  1856.0  palmeiras
       KEY   tempo            url
1   691459  1912.0    corinthians
2    43452   464.0      sao-paulo
3  2760594   736.0    corinthians
6  2131163   464.0    ponte-preta
7  3260615  1357.0  internacional


In [8]:
#AGRUPANDO TIMES E RESPECTIVOS TEMPOS GASTOS NA PAGINA DAQUELE TIME
df_con_train_simple = df_con_train_proc_simple.groupby(by=('KEY','url')).sum()
df_con_test_simple = df_con_test_proc_simple.groupby(by=('KEY','url')).sum()
print(df_con_train_simple.tail(25))
print(df_con_test_simple.tail(25))
df_con_test_simple.query('KEY == 3412351')

                          tempo
KEY     url                    
3414419 fluminense       1235.0
        gremio           9809.0
        internacional    1429.0
        palmeiras        3259.0
        santos           2351.0
        vasco             776.0
3414420 corinthians      1035.0
        internacional    1696.0
        sao-paulo        6087.0
3414421 atletico-mg      9956.0
        botafogo         1357.0
        chapecoense      7662.0
        corinthians      5310.0
        cruzeiro       140507.0
        flamengo         4956.0
        gremio           1169.0
        palmeiras        5326.0
        santos          10410.0
        sao-paulo        1442.0
        vasco            3423.0
3414422 corinthians       464.0
        flamengo         8009.0
        fluminense        736.0
        sao-paulo         970.0
        vasco             464.0
                         tempo
KEY     url                   
3414348 sao-paulo       2904.0
        sport            729.0
        vasc

tempo
KEY     url                   
3412351 atletico-mg     2525.0
        corinthians     1173.0
        cruzeiro        8051.0
        flamengo        1361.0
        fluminense      3934.0
        gremio           349.0
        internacional    317.0
        palmeiras      14970.0
        sport           8319.0
        vasco            848.0
        vitoria         7407.0

In [9]:
df_con_test_simple.to_csv(save_path+'/20180924_con_test_sampled_simple.csv')
df_con_train_simple.to_csv(save_path+'/20180924_con_train_sampled_simple.csv')